# Import packages

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection i8mport train_test_split
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Get the Data

In [ ]:
df = pd.read_csv('yellow_tripdata_2021-03.csv') # Load tha dataset
df

### Take a Quick Look at the Data Structure

In [ ]:
df.shape

In [ ]:
df.sample(5)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## Data Splitting

In [ ]:
df_train , df_test = train_test_split(df , test_size=.2, random_state=10)

In [ ]:
df_train, df_val = train_test_split(df_train, test_size=.25, random_state=5)

## Cleaning Training/ Validation / Test Data 

In [ ]:
df_train.shape

In [ ]:
df_val.shape

In [ ]:
df_test.shape

In [ ]:
nulls = df_train.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"] 
nulls

In [ ]:
df_train = df_train.dropna(how ='any')

In [ ]:
nulls = df_train.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"] 
nulls

In [ ]:
#----------------- df_val ---------------------#

In [ ]:
nulls = df_val.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"]
nulls

In [ ]:
df_val = df_val.dropna(how ='any')

In [ ]:
nulls = df_val.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"]
nulls

In [ ]:
#----------------- df_test ---------------------#

In [ ]:
nulls = df_test.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"]
nulls

In [ ]:
df_test = df_test.dropna(how ='any')

In [ ]:
nulls = df_test.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"]
nulls

#### In this step, it took the hours pick up and days of capture and put them in new columns

In [ ]:
##--------------------- train ---------------------## 

In [ ]:
df_train['Day'] = pd.to_datetime(df_train['tpep_pickup_datetime']).dt.day

In [ ]:
df_train['Hour_pickup'] = pd.to_datetime(df_train['tpep_pickup_datetime']).dt.hour
df_train

In [ ]:
df1_train= df_train.drop(df_train[df_train.Day < 20 ].index)
df1_train

In [ ]:
df1_train['Day'] = pd.to_datetime(df_train['tpep_pickup_datetime']).dt.weekday

In [ ]:
##----------------------------- val ----------------------##

In [ ]:
df_val['Day'] = pd.to_datetime(df_val['tpep_pickup_datetime']).dt.day

In [ ]:
df_val['Hour_pickup'] = pd.to_datetime(df_val['tpep_pickup_datetime']).dt.hour
df_val

In [ ]:
df1_val= df_val.drop(df_val[df_val.Day < 20 ].index)
df1_val

In [ ]:
df1_val['Day'] = pd.to_datetime(df_val['tpep_pickup_datetime']).dt.weekday

In [ ]:
##----------------------------- Test ----------------------##

In [ ]:
df_test['Day'] = pd.to_datetime(df_test['tpep_pickup_datetime']).dt.day

In [ ]:
df_test['Hour_pickup'] = pd.to_datetime(df_test['tpep_pickup_datetime']).dt.hour
df_test

In [ ]:
df1_test= df_test.drop(df_test[df_test.Day < 20 ].index)
df1_test

#### In this step, delete some columns the unimportant 

In [ ]:
#----------------- df_train ---------------------#

In [ ]:
df1_train.drop(['store_and_fwd_flag', 'PULocationID','DOLocationID','tpep_pickup_datetime','tpep_dropoff_datetime']
         ,axis=1, inplace=True ) # delete four columns ,(axis=1:refers to column dimension)

In [ ]:
#----------------- df_val ---------------------#

In [ ]:
df1_val.drop(['store_and_fwd_flag', 'PULocationID','DOLocationID','tpep_pickup_datetime','tpep_dropoff_datetime']
            ,axis=1, inplace=True ) # delete four columns ,(axis=1:refers to column dimension)

In [ ]:
#----------------- df_test ---------------------#

In [ ]:
df1_test.drop(['store_and_fwd_flag', 'PULocationID','DOLocationID','tpep_pickup_datetime','tpep_dropoff_datetime']
         ,axis=1, inplace=True ) # delete four columns ,(axis=1:refers to column dimension)

#### Delete some duplicate values

In [ ]:
duplicate = df1_train.duplicated()
print(duplicate.sum())
df1_train[duplicate]

In [ ]:
df1_train.drop_duplicates(inplace=True)

In [ ]:
duplicate = df1_train.duplicated()
print(duplicate.sum())
df1_train[duplicate]

In [ ]:
#----------------- df_val ---------------------#

In [ ]:
duplicate = df1_val.duplicated()
print(duplicate.sum())
df1_val[duplicate]

In [ ]:
df1_val.drop_duplicates(inplace=True)

In [ ]:
duplicate = df1_val.duplicated()
print(duplicate.sum())
df1_val[duplicate]

In [ ]:
#----------------- df_test ---------------------

In [ ]:
duplicate = df1_test.duplicated()
print(duplicate.sum())
df1_test[duplicate]

In [ ]:
df1_test.drop_duplicates(inplace=True)

In [ ]:
duplicate = df1_test.duplicated()
print(duplicate.sum())
df1_test[duplicate]

In [ ]:
Day_hours = df1_train.groupby('Hour_pickup').Day.value_counts().sort_values(ascending=False)
Day_hours

In [ ]:
#df1_train.copy()

### Looking for Correlations

In [ ]:
df1_train.corr()

In [ ]:
train_corr = df1_train.corr()

In [ ]:
train_corr["fare_amount"].sort_values(ascending=False)

# visualization for train

In [ ]:
plt.figure(figsize =[15,15])
sns.heatmap(df1_train.corr(),cmap="seismic",annot=True, vmin=-1, vmax=1, linewidths=.5);
plt.savefig('plot15.png', dpi = 400, bbox_inches = 'tight');

In [ ]:
sns.pairplot(df1_train)

In [ ]:
df1_train.plot(kind="scatter", x="trip_distance", y="fare_amount");

In [ ]:
df1_train.plot(kind="scatter", x="fare_amount", y="tolls_amount");

In [ ]:
df1_train.plot(kind="scatter", x="tip_amount", y="tolls_amount", alpha=0.1);

In [ ]:
attributes = ["fare_amount", "trip_distance", "tolls_amount",
              "tip_amount","improvement_surcharge"]
scatter_matrix(df1_train[attributes], figsize=(12, 8));

In [ ]:
 ##------------------ End of Code Train -----------------##

# Building and Evaluating the Model

In [ ]:
#spilt df_train to X and y

In [ ]:
X , y = df1_train[['trip_distance','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge']], df1_train['fare_amount']

### 1. Liner Regression

In [ ]:
#--------------------- train -----------------#

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(X,y)

In [ ]:
lr.score(X,y)

In [ ]:
#--------------------- val -----------------#

In [ ]:
X_val , y_val = df1_val[['trip_distance','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge']], df1_val['fare_amount']

# ***only fit on traininggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggg***

In [ ]:
lr.score(X_val,y_val)

In [ ]:
y_pred = lr.predict(X_val)

### Model after improvement

In [ ]:
##--------------------- train -----------------

In [ ]:
df1_train['RatecodeID'] = df1_train.RatecodeID.astype('category')
df1_train['RatecodeID']

In [ ]:
df1_train.info()

In [ ]:
df_dummies_train = pd.get_dummies(data=df1_train)
df_dummies_train.head()

In [ ]:
X , y = df_dummies_train[['trip_distance','RatecodeID_1.0','RatecodeID_2.0','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge','Hour_pickup','Day']], df_dummies_train['fare_amount']

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(X,y)

In [ ]:
lr.score(X,y)

In [ ]:
#--------------------- Val dummies -----------------

In [ ]:
df1_val['RatecodeID'] = df1_val.RatecodeID.astype('category')
df1_val['RatecodeID']

In [ ]:
df_dummies_val = pd.get_dummies(data=df1_val)
df_dummies_val.head()

In [ ]:
X_val , y_val = df_dummies_val[['trip_distance','RatecodeID_1.0','RatecodeID_2.0','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge','Hour_pickup','Day']], df_dummies_val['fare_amount']

In [ ]:
lr.score(X_val,y_val)

In [ ]:
y_pred = lr.predict(X_val)

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.scatter(y_val, y_pred)
ax.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
plt.savefig('plot15.png', dpi = 400, bbox_inches = 'tight');
plt.show()

In [ ]:
#--------------------- test -----------------

In [ ]:
df1_test.info()

In [ ]:
df1_test['RatecodeID'] = df1_test.RatecodeID.astype('category')
df1_test['RatecodeID']

In [ ]:
df_dummies_test = pd.get_dummies(data=df1_test)
df_dummies_test.head()

In [ ]:
X_test , y_test = df_dummies_test[['trip_distance','RatecodeID_1.0','RatecodeID_2.0','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge','Hour_pickup','Day']], df_dummies_test['fare_amount']

In [ ]:
lr.score(X_test,y_test)

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.scatter(y_test, y_pred_test)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
plt.show()

### Linear Polynomial

In [ ]:
poly_features = PolynomialFeatures(degree= 2)
# transforms the existing features to higher degree features.
X_train_poly = poly_features.fit_transform(X)
# fit the transformed features to Linear Regression
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y)

In [ ]:
#--------------------- train -----------------

In [ ]:
# predicting on training data-set
y_train_predicted = poly_model.predict(X_train_poly)
# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y, y_train_predicted))
r2_train = r2_score(y, y_train_predicted)
print('rmse = ',rmse_train)
print('r2 = ',r2_train)

In [ ]:
#--------------------- val -----------------

In [ ]:
# predicting on val data-set
y_test_predict = poly_model.predict(poly_features.fit_transform(X_test))
# evaluating the model on val dataset
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_predict))
r2_test = r2_score(y_test, y_test_predict)
print('rmse = ',rmse_test)
print('r2 = ',r2_test)

 ### Ridge

In [ ]:
#--------------------- train -----------------#

In [ ]:
rr = Ridge(alpha=0.1)
rr.fit(X, y)
pred_train_rr= rr.predict(X)
print(np.sqrt(mean_squared_error(y,pred_train_rr)))
print(r2_score(y, pred_train_rr))

In [ ]:
#--------------------- val -----------------#

In [ ]:
pred_val_rr= rr.predict(X_val)
print(np.sqrt(mean_squared_error(y_val,pred_val_rr)))
print(r2_score(y_val, pred_val_rr))

In [ ]:
pred_test_rr= rr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_rr)))
print(r2_score(y_test, pred_test_rr))

In [ ]:
#--------------------- train -----------------

In [ ]:
model_lasso = Lasso(alpha=0.01)
model_lasso.fit(X, y)
pred_train_lasso= model_lasso.predict(X)
print(np.sqrt(mean_squared_error(y,pred_train_lasso)))
print(r2_score(y, pred_train_lasso))

In [ ]:
#--------------------- val -----------------

In [ ]:
pred_val_lasso= model_lasso.predict(X_val)
print(np.sqrt(mean_squared_error(y_val,pred_val_lasso)))
print(r2_score(y_val, pred_val_lasso))

In [ ]:
pred_test_lasso= model_lasso.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_lasso)))
print(r2_score(y_test, pred_test_lasso))